In [104]:
# -*- coding: utf8 -*-
from os.path import expanduser
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K1"
repertoire="/Users/gilles/Dropbox/Partage-Broadway/GAM/Caches/NosCaches/Kalabas/K400"
serie=repertoire+"/"
#########################IMPORTS############################################
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml
import ParFuMor as PFM
from ParFuMor import *
import pickle
from cellbell import ding

In [105]:
debug=0
print_glose=True
print_cloze=True
print_radicaux=True
cloze_expanded=True
numeros={'1':'Un','2':'Deux','3':'Trois','4':'Quatre','5':'Cinq'}
personnes={'1sg':'UnSg','2sg':'DeuxSg','3sg':'TroisSg','1du':'UnDu','2du':'DeuxDu','3du':'TroisDu','1pl':'UnPl','2pl':'DeuxPl','3pl':'TroisPl'}

commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}\\grapho{%s}}"
#commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}{\\dn %s}}"
#commandGrapho=u"\\newcommand{\\%s}{\\strutgb{0pt}{%s}}"
commandPhono=u"\\newcommand{\\%sP}{\\textipa{%s}}"
commandGlose=u"\\newcommand{\\%sG}{\\textGlose{%s}}"

In [106]:
with open(serie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)
with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.load(stream)
with open(serie+"MorphoSyntax.yaml", 'r') as stream:
    morphosyntax=yaml.load(stream)
with open(serie+"Hierarchie-S2.pkl", 'rb') as input:
   PFM.hierarchieCF = pickle.load(input)
with open(serie+"Lexique-S2.pkl", 'rb') as input:
   PFM.lexique = pickle.load(input)
with open(serie+"Regles-S2.pkl", 'rb') as input:
   PFM.regles = pickle.load(input)

In [107]:
PFM.lexique.lexemes[u'dans']

case PREP:	jet	dans	jet	\textRadical{jet}	radical: jet	racine:  


jet, PREP, DANS
		jet :
			PREP:	jet	dans	jet	\textRadical{jet}	radical: jet	racine: 

#### Définition des segments

In [108]:
consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]
gabarits=phonology["gabarits"]
derives=phonology["derives"]
nom_classe=phonology["nom_classe"]
nom_apo=phonology["apophonies"]
nom_mut=phonology["mutations"]
syllabes=phonology["syllabes"]

#### Attention aux correspondances pour les syllabes
YAML interprète la clé no comme False

In [109]:
def parse_grapho(graphie):
    def change_monograms(chaine):
        result=""
        for monogramme in chaine:
            if monogramme in monogrammes:
                result+=monogrammes[monogramme]
            else:
                result+=monogramme
        return result

    def change_bigrams(chaine):
        result=change_monograms(chaine)
        for bigramme in bigrammes:
            result=result.replace(bigramme,bigrammes[bigramme])
        return result

    
    chunks=re.findall(ur"\s+|[ptkqbdgmnNfTsSvDzZjwRrlLyvWJ]?[aeiouAEIUO]?", graphie,re.U)
    result=[]
    if debug or "m" in graphie: print graphie
    for chunk in chunks:
        if debug or "m" in graphie: print chunk,
        #ATTENTION : YAML interprète "no" comme FALSE
        if chunk in syllabes.keys():
            result.append(syllabes[chunk])
        else:
            result.append(chunk)
    if debug or "m" in graphie: 
        print result
        print
    return change_bigrams("".join(result))

In [110]:
def recoder(chaine,table,boolUTF8=True):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    if boolUTF8:
        return result
    else:
        return result.encode("utf8")
#translit=string.maketrans(u'iueoaftgzZvjkSpN',u'tgazpHTGZJVkXyxI')

In [111]:
phonoIn =  unicode(phonology["translations"]["grapho"]["in"])
graphoIn = [ord(char) for char in phonoIn]
graphoOut = unicode(phonology["translations"]["grapho"]["out"])
translit = dict(zip(graphoIn, graphoOut))
if "monogrammes" in phonology["translations"]:
    monogrammes = phonology["translations"]["monogrammes"]
else:
    monogrammes = {}
bigrammes = phonology["translations"]["bigrammes"]


In [112]:
accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

deligatures=phonology["translations"]["deligatures"]

In [113]:
#translit

In [114]:
tipaIn = unicode(phonology["translations"]["ipa"]["in"])
ipaIn = [ord(char) for char in tipaIn]
ipaOut = unicode(phonology["translations"]["ipa"]["out"])
toipa = dict(zip(ipaIn, ipaOut))

In [115]:
parse_grapho(u"SugiNabUg")

u'Sugi.sabUg'

In [116]:
def parse_cloze(glose):
    if debug: print [glose]
    chunks=re.findall(r"\\cacherGloses{([^}]*)?}|(\w+)", glose,re.UNICODE)
    result=[]
    for chunk in chunks:
        result.extend([x for x in chunk if x!=""])
    return (u"%s;"%len(result)+";".join(result),"".join(result))

In [117]:
if "Cas" in morphosyntax:
    casSyntagmes=morphosyntax["Cas"]
else:
    casSyntagmes=""
lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
casPreposition={}
for preposition in lexiquePrepositions:
    prep=preposition.upper()
    if casSyntagmes and prep in casSyntagmes:
        casPreposition[preposition]=casSyntagmes[prep]
    elif casSyntagmes and "PREP" in casSyntagmes:
        casPreposition[preposition]=casSyntagmes["PREP"]
    else:
        casPreposition[preposition]=""
glosePREP={x:u"%s%s"%(x.upper(),cacherGloses("["+casPreposition[x].strip("+").capitalize()+"]")) for x in casPreposition if "+" in casPreposition[x]}

In [118]:
glosePREP

{'ENTRE': u'ENTRE\\cacherGloses{[Obl]}',
 u'apr\xe8s': u'APR\xc8S\\cacherGloses{[Obl]}',
 'avec': u'AVEC\\cacherGloses{[Abs]}',
 'dans': u'DANS\\cacherGloses{[Obl]}',
 'de': u'DE\\cacherGloses{[Abs]}',
 u'derri\xe8re': u'DERRI\xc8RE\\cacherGloses{[Obl]}',
 'devant': u'DEVANT\\cacherGloses{[Obl]}',
 'pour': u'POUR\\cacherGloses{[Abs]}',
 'sous': u'SOUS\\cacherGloses{[Obl]}',
 'sur': u'SUR\\cacherGloses{[Obl]}',
 'vers': u'VERS\\cacherGloses{[Obl]}',
 u'\xe0': u'\xc0\\cacherGloses{[Abs]}'}

In [119]:
try:
    __IPYTHON__ 
    ipython=True
except: 
    ipython=False

version=os.path.basename("__file__")
time_stamp='%s' % time.strftime("%y%m%d-%H%M")
print "%% version : "+version
print "%% traitement : "+time_stamp

if ipython or True:
#    lexeme_nom="lexemes.txt"
#    phrase_nom="phrases.txt"
    pass
else:
    parser=optparse.OptionParser()
    parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
    parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
    parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
    parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

    (options, args) = parser.parse_args()
    lexeme_nom=args[0]
    phrase_nom=args[1]

%% version : __file__
%% traitement : 191124-0508


In [120]:
tableaux={}
gloseClozes={}
declarations=[]
for categorie in PFM.lexique.catLexeme:
    if not categorie in tableaux:
        tableaux[categorie]=set()
        gloseClozes[categorie]=[]
    if verbose: print categorie
    for lexeme in PFM.lexique.catLexeme[categorie]:
        if verbose: print PFM.lexique.lexemes[lexeme]
        for case in PFM.lexique.lexemes[lexeme].paradigme.cases:
            caseGlose=case.glose
            if categorie in PFM.categoriesMajeures:
                nom=PFM.lexique.lexemes[lexeme].nom
            else:
                nom=PFM.lexique.lexemes[lexeme].nom.upper()
                gloseCase=case.glose
                if categorie=="PREP" and gloseCase in glosePREP:
                    caseGlose=glosePREP[gloseCase]
                elif categorie=="PREP":
                    caseGlose=caseGlose.upper()
#                print caseGlose
            ref=PFM.modifierGlose(nom,case.sigma,"ref")
            ref=recoder(ref,deaccent)
            for ligature in deligatures:
                ref=ref.replace(ligature,deligatures[ligature])
            for pers in personnes:
                ref=ref.replace(pers,personnes[pers])
            for num in numeros:
                ref=ref.replace(num,numeros[num])
#            phono=case.forme
#            print ("lexeme",lexeme)
            grapho=chaine2utf8(re.sub(ur"\s+",ur"",parse_grapho(recoder(case.forme,translit))))
            if print_radicaux:
                phono=case.detoure.replace(" ","")
            else:
                phono=case.forme.replace(" ","")
#            print [ref,grapho]
            declarations.append(commandGrapho%(ref,grapho))
            declarations.append(commandPhono%(ref,phono))
            declarations.append(commandGlose%(ref,caseGlose))
            if print_glose:
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\"+ref+"G \\\\")
            else:
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\blanc{\\"+ref+"G} \\\\")
            if print_cloze:
                vedette=nom.split(".")[0]
                gloses,strGlose=parse_cloze(case.glose)
                phono=case.forme.replace(" ","")
                if debug: print [grapho, recoder(phono,toipa)]
                if cloze_expanded:
                    try:
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),recoder(case.decoupe,toipa),
                                         case.sigma,
                                         strGlose,
                                         gloses])
                    except NameError:
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),
                                         strGlose,
                                         gloses])
                else:
                    try:
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),recoder(case.decoupe,toipa),case.sigma,gloses])
                    except NameError:
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),gloses])
                gloseClozes[categorie].append(cloze)
    

dimagabbug
di ma ga b bu g  [u'di', u'ma', u'ga', u'b', u'bu', u'g', u'']

dimagabbu
di ma ga b bu  [u'di', u'ma', u'ga', u'b', u'bu', u'']

dimagabbuz
di ma ga b bu z  [u'di', u'ma', u'ga', u'b', u'bu', u'z', u'']

dimagabbuD
di ma ga b bu D  [u'di', u'ma', u'ga', u'b', u'bu', u'D', u'']

dimaggibeg
di ma g gi be g  [u'di', u'ma', u'g', u'gi', u'be', u'g', u'']

dimaggibe
di ma g gi be  [u'di', u'ma', u'g', u'gi', u'be', u'']

dimaggibez
di ma g gi be z  [u'di', u'ma', u'g', u'gi', u'be', u'z', u'']

dimaggibeD
di ma g gi be D  [u'di', u'ma', u'g', u'gi', u'be', u'D', u'']

dimagabpig
di ma ga b pi g  [u'di', u'ma', u'ga', u'b', u'pi', u'g', u'']

dimagabpi
di ma ga b pi  [u'di', u'ma', u'ga', u'b', u'pi', u'']

dimagabpiz
di ma ga b pi z  [u'di', u'ma', u'ga', u'b', u'pi', u'z', u'']

dimagabpiD
di ma ga b pi D  [u'di', u'ma', u'ga', u'b', u'pi', u'D', u'']

magabbufug
ma ga b bu fu g  [u'ma', u'ga', u'b', u'bu', u'fu', u'g', u'']

magabbufu
ma ga b bu fu  [u'ma', u'ga', u'b', u'bu',

nu do r ra de mu g  [u'nu', u'do', u'r', u'ra', u'de', u'mu', u'g', u'']

nudorrademu
nu do r ra de mu  [u'nu', u'do', u'r', u'ra', u'de', u'mu', u'']

nudorrademuz
nu do r ra de mu z  [u'nu', u'do', u'r', u'ra', u'de', u'mu', u'z', u'']

nudorrademuD
nu do r ra de mu D  [u'nu', u'do', u'r', u'ra', u'de', u'mu', u'D', u'']

dum
du m  [u'du', u'm', u'']

medi
me di  [u'me', u'di', u'']

medu
me du  [u'me', u'du', u'']

medi
me di  [u'me', u'di', u'']

kum
ku m  [u'ku', u'm', u'']

meki
me ki  [u'me', u'ki', u'']

meku
me ku  [u'me', u'ku', u'']

meki
me ki  [u'me', u'ki', u'']

rum
ru m  [u'ru', u'm', u'']

meri
me ri  [u'me', u'ri', u'']

meru
me ru  [u'me', u'ru', u'']

meri
me ri  [u'me', u'ri', u'']

titbime
ti t bi me  [u'ti', u't', u'bi', u'me', u'']

titbima
ti t bi ma  [u'ti', u't', u'bi', u'ma', u'']

titbimi
ti t bi mi  [u'ti', u't', u'bi', u'mi', u'']

titbimu
ti t bi mu  [u'ti', u't', u'bi', u'mu', u'']

titbimeb
ti t bi me b  [u'ti', u't', u'bi', u'me', u'b', u'']

titbimab

qo bu Di mi  [u'qo', u'bu', u'Di', u'mi', u'']

buDimiru
bu Di mi ru  [u'bu', u'Di', u'mi', u'ru', u'']

buDimiza
bu Di mi za  [u'bu', u'Di', u'mi', u'za', u'']

nebuDimiZa
ne bu Di mi Za  [u'ne', u'bu', u'Di', u'mi', u'Za', u'']

DozokemDu
Do zo ke m Du  [u'Do', u'zo', u'ke', u'm', u'Du', u'']

zokemDuru
zo ke m Du ru  [u'zo', u'ke', u'm', u'Du', u'ru', u'']

zokemDuza
zo ke m Du za  [u'zo', u'ke', u'm', u'Du', u'za', u'']

zokemDuZe
zo ke m Du Ze  [u'zo', u'ke', u'm', u'Du', u'Ze', u'']

qozikmemo
qo zi k me mo  [u'qo', u'zi', u'k', u'me', u'mo', u'']

zikmemoru
zi k me mo ru  [u'zi', u'k', u'me', u'mo', u'ru', u'']

zikmemoza
zi k me mo za  [u'zi', u'k', u'me', u'mo', u'za', u'']

nezikmemoZa
ne zi k me mo Za  [u'ne', u'zi', u'k', u'me', u'mo', u'Za', u'']

qozokimi
qo zo ki mi  [u'qo', u'zo', u'ki', u'mi', u'']

zokimiru
zo ki mi ru  [u'zo', u'ki', u'mi', u'ru', u'']

zokimiza
zo ki mi za  [u'zo', u'ki', u'mi', u'za', u'']

nezokimiZa
ne zo ki mi Za  [u'ne', u'zo', u'ki', u'mi', u'

In [121]:
if print_radicaux:
    nomDeclaration="Declarations-Radicaux.tex"
    nomTableaux="Tableaux-Radicaux.yaml"
else:
    nomDeclaration="Declarations.tex"
    nomTableaux="Tableaux.yaml"

In [122]:
with codecs.open(serie+nomDeclaration, 'wb', encoding='utf8') as output:
    for declaration in declarations:
#        print type(declaration),declaration
        output.write(declaration+"\n")

In [123]:
ding()

In [124]:
if print_cloze:
    with codecs.open(serie+"Clozes.txt", 'wb', encoding='utf8') as output:
        for categorie in gloseClozes:
            output.write("#\t"+categorie+"\n#\n#\n")
            for cloze in gloseClozes[categorie]:
                output.write(cloze+"\n")
            output.write("#\n#\n#\n")

In [125]:
yaml.safe_dump(tableaux, file(serie+nomTableaux, 'w'), encoding='utf-8', allow_unicode=True)

In [126]:
#gloseClozes

In [127]:
[l for l in declarations if u"Prep" in l]

[]

In [128]:
chaine="pumak"
print "parse", parse_grapho(chaine)
print "parse, recoder",recoder(parse_grapho(chaine),translit)
print "recoder", recoder(chaine,translit)
print "recoder, parse",parse_grapho(recoder(chaine,translit))


parse pumak
pu ma k  ['pu', 'ma', 'k', '']

pumak
parse, recoder pumak
pu ma k  ['pu', 'ma', 'k', '']

genid
recoder genid
recoder, parse genid


In [129]:
DG=ur"\offrirVdPstTroisSgCP{}    \DEFSgBDatP{}   \LilouBSgDatP{}   \INDSgCAbsP{}   \sanglierCSgAbsP{}   \DEFSgDErgP{}   \trappeurDSgErgP{}"
print " ".join([m for m in DG.split(" ") if m!=""][::-1])

\trappeurDSgErgP{} \DEFSgDErgP{} \sanglierCSgAbsP{} \INDSgCAbsP{} \LilouBSgDatP{} \DEFSgBDatP{} \offrirVdPstTroisSgCP{}
